In [ ]:
import sys
sys.path.append("../../..")

import os

from impit.framework.client import IMPITClient
from impit.segmentation.tests.pull_test_data import get_bronchus_data

from loguru import logger

host = "127.0.0.1"
port = 8001

api_key = "018f9df3-43f7-4efc-9cd2-43c21f0c9556"

algorithm_name = 'Bronchus Segmentation'

log_level = "INFO" # Choose an appropriate level of logging output: "DEBUG" or "INFO"

logger.remove()
handler_id = logger.add(sys.stderr, level=log_level)

In [ ]:
images = get_bronchus_data()

In [ ]:
client = IMPITClient(host, port, api_key, algorithm_name)

In [ ]:
orthanc_location = client.add_dicom_location(name='Orthanc',
           host= '127.0.0.1',
           port= 4242,
           ae_title= 'ORTHANC')

In [ ]:
dataset = client.add_dataset()

In [ ]:
pat_id = list(images.keys())[0]
ct_file = images[pat_id]
data_object = client.add_data_object(dataset, file_path=ct_file)

In [ ]:
algorithm = client.get_default_settings()

In [ ]:
client.get_dataset(dataset)

In [ ]:
for status in client.run_algorithm(dataset):
    print(status)

In [ ]:
output_directory = os.path.join(".", "results", pat_id)
client.download_output_objects(dataset, output_path=output_directory)